In [ ]:
# 간단한 바운딩 박스 수정 - 핵심 문제만 해결

import torch
import cv2 as cv
from ultralytics import YOLO
import numpy as np
from pathlib import Path
import random
from datetime import datetime

class SimpleDataConverter:
    """간단한 데이터 변환기 - 핵심 문제만 해결"""
    
    def __init__(self, source_path="trashnet", target_path="dataset"):
        self.source_path = Path(source_path)
        self.target_path = Path(target_path)
        
        self.class_mapping = {
            'glass': 0,
            'metal': 1,
            'paper': 2,
            'plastic': 3,
            'trash': 4
        }
        
        # 폴더 생성
        for split in ["train", "val"]:
            (self.target_path / "images" / split).mkdir(parents=True, exist_ok=True)
            (self.target_path / "labels" / split).mkdir(parents=True, exist_ok=True)
    
    def process_single_image(self, img_path, class_id, split):
        """이미지 처리 - 객체 중심 영역만 바운딩 박스로"""
        try:
            image = cv.imread(str(img_path))
            if image is None:
                return False
            
            h, w = image.shape[:2]
            
            # 새 파일명
            new_filename = f"{class_id}_{img_path.stem}_{datetime.now().strftime('%H%M%S')}.jpg"
            
            # 이미지 저장
            target_img_path = self.target_path / "images" / split / new_filename
            cv.imwrite(str(target_img_path), image)
            
            # 🔥 핵심 수정: 전체가 아닌 중심 80% 영역을 바운딩 박스로
            # 이전: 전체 이미지 (center_x=0.5, center_y=0.5, width=1.0, height=1.0)
            # 수정: 중심 80% 영역 (배경 제외)
            center_x = 0.5      # 중심은 그대로
            center_y = 0.5      # 중심은 그대로  
            width = 0.8         # 가로 80%
            height = 0.8        # 세로 80%
            
            # 라벨 파일 저장
            label_filename = f"{new_filename.rsplit('.', 1)[0]}.txt"
            label_path = self.target_path / "labels" / split / label_filename
            
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {center_x} {center_y} {width} {height}\n")
            
            return True
            
        except Exception as e:
            print(f"❌ 처리 실패 {img_path}: {e}")
            return False
    
    def convert_data(self, train_ratio=0.8):
        """데이터 변환 실행"""
        print("🔄 간단 바운딩 박스 변환 시작...")
        
        total = 0
        
        for folder_name, class_id in self.class_mapping.items():
            folder_path = self.source_path / folder_name
            
            if not folder_path.exists():
                continue
            
            # 이미지 찾기
            images = list(folder_path.glob("*.jpg")) + list(folder_path.glob("*.png"))
            
            if not images:
                continue
            
            print(f"📁 {folder_name}: {len(images)}장 변환 중...")
            
            # 랜덤 섞기 및 분할
            random.shuffle(images)
            split_idx = int(len(images) * train_ratio)
            
            train_images = images[:split_idx]
            val_images = images[split_idx:]
            
            # 변환
            for img_path in train_images:
                self.process_single_image(img_path, class_id, "train")
            
            for img_path in val_images:
                self.process_single_image(img_path, class_id, "val")
            
            total += len(images)
            print(f"✅ {folder_name} 완료")
        
        print(f"🎉 총 {total}장 변환 완료!")


class SimpleLabeler:
    """간단한 마우스 라벨러 - 복잡한 기능 제거"""
    
    def __init__(self, dataset_path="dataset"):
        self.dataset_path = Path(dataset_path)
        self.current_image = None
        self.current_image_path = None
        self.current_label_path = None
        self.bbox = None  # 하나의 박스만
        self.drawing = False
        self.start_point = None
        
        self.image_files = list((self.dataset_path / "images" / "train").glob("*.jpg"))
        self.current_index = 0
        
        self.class_names = ['glass', 'metal', 'paper', 'plastic', 'trash']
        
        print(f"📝 {len(self.image_files)}개 이미지 라벨링 준비")
    
    def mouse_callback(self, event, x, y, flags, param):
        """마우스 콜백"""
        if event == cv.EVENT_LBUTTONDOWN:
            self.drawing = True
            self.start_point = (x, y)
        
        elif event == cv.EVENT_LBUTTONUP:
            if self.drawing and self.start_point:
                self.drawing = False
                
                x1, y1 = self.start_point
                x2, y2 = x, y
                
                # 좌표 정렬
                if x1 > x2: x1, x2 = x2, x1
                if y1 > y2: y1, y2 = y2, y1
                
                # 파일명에서 클래스 추출
                class_id = int(self.current_image_path.name.split('_')[0])
                
                self.bbox = (class_id, x1, y1, x2, y2)
                print(f"✅ 박스 설정: {self.class_names[class_id]} ({x1},{y1}) to ({x2},{y2})")
                self.draw_current_state()
    
    def draw_current_state(self):
        """화면에 그리기"""
        if self.current_image is None:
            return
        
        img_display = self.current_image.copy()
        
        # 박스 그리기
        if self.bbox:
            class_id, x1, y1, x2, y2 = self.bbox
            cv.rectangle(img_display, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv.putText(img_display, f"{self.class_names[class_id]}", (x1, y1-10),
                      cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # 정보 표시
        info = f"[{self.current_index + 1}/{len(self.image_files)}]"
        cv.putText(img_display, info, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        instructions = "Mouse: Draw Box | SPACE: Save | N: Next | Q: Quit"
        cv.putText(img_display, instructions, (10, img_display.shape[0] - 20), 
                  cv.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        cv.imshow('Simple Labeler', img_display)
    
    def save_label(self):
        """라벨 저장"""
        if not self.bbox:
            print("⚠️ 박스가 없습니다.")
            return
        
        h, w = self.current_image.shape[:2]
        class_id, x1, y1, x2, y2 = self.bbox
        
        # YOLO 형식으로 변환
        center_x = (x1 + x2) / (2 * w)
        center_y = (y1 + y2) / (2 * h)
        width = (x2 - x1) / w
        height = (y2 - y1) / h
        
        with open(self.current_label_path, 'w') as f:
            f.write(f"{class_id} {center_x:.6f} {center_y:.6f} {width:.6f} {height:.6f}\n")
        
        print("💾 저장 완료!")
    
    def load_current_image(self):
        """현재 이미지 로드"""
        if self.current_index >= len(self.image_files):
            return
        
        self.current_image_path = self.image_files[self.current_index]
        self.current_image = cv.imread(str(self.current_image_path))
        
        # 라벨 파일 경로
        label_name = self.current_image_path.stem + ".txt"
        self.current_label_path = self.dataset_path / "labels" / "train" / label_name
        
        # 기존 라벨 로드
        self.bbox = None
        if self.current_label_path.exists():
            try:
                h, w = self.current_image.shape[:2]
                with open(self.current_label_path, 'r') as f:
                    line = f.readline().strip()
                    if line:
                        parts = line.split()
                        if len(parts) == 5:
                            class_id = int(parts[0])
                            center_x = float(parts[1]) * w
                            center_y = float(parts[2]) * h
                            width = float(parts[3]) * w
                            height = float(parts[4]) * h
                            
                            x1 = int(center_x - width/2)
                            y1 = int(center_y - height/2)
                            x2 = int(center_x + width/2)
                            y2 = int(center_y + height/2)
                            
                            self.bbox = (class_id, x1, y1, x2, y2)
            except:
                pass
        
        self.draw_current_state()
        print(f"📁 로드: {self.current_image_path.name}")
    
    def start_labeling(self):
        """라벨링 시작"""
        if not self.image_files:
            print("❌ 이미지가 없습니다.")
            return
        
        cv.namedWindow('Simple Labeler')
        cv.setMouseCallback('Simple Labeler', self.mouse_callback)
        
        self.load_current_image()
        
        while True:
            key = cv.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            elif key == ord(' '):  # 저장
                self.save_label()
            elif key == ord('n'):  # 다음
                self.current_index = (self.current_index + 1) % len(self.image_files)
                self.load_current_image()
        
        cv.destroyAllWindows()
        print("✅ 라벨링 완료!")


def create_yaml():
    """YAML 파일 생성"""
    yaml_content = """path: ./dataset
train: images/train
val: images/val
nc: 5
names: ['glass', 'metal', 'paper', 'plastic', 'trash']
"""
    
    with open("dataset.yaml", "w") as f:
        f.write(yaml_content)
    print("✅ dataset.yaml 생성 완료")


def main():
    print("🎯 간단한 바운딩 박스 수정")
    print("=" * 40)
    
    while True:
        print("\n선택하세요:")
        print("1️⃣ 데이터 변환 (80% 바운딩 박스)")
        print("2️⃣ 마우스로 바운딩 박스 수정")  
        print("3️⃣ YAML 파일 생성")
        print("0️⃣ 종료")
        
        choice = input("선택: ").strip()
        
        if choice == "1":
            converter = SimpleDataConverter()
            converter.convert_data()
        
        elif choice == "2":
            labeler = SimpleLabeler()
            labeler.start_labeling()
        
        elif choice == "3":
            create_yaml()
        
        elif choice == "0":
            break
        
        else:
            print("❌ 잘못된 선택")


if __name__ == "__main__":
    main()